# PAINTINGS AI

In [1]:
# Carga librerías
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
# Transformaciones
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Dataset personalizado
class ArtDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.authors = sorted(self.data['artist'].unique())
        self.styles = sorted(self.data['style'].unique())
        self.author_to_idx = {author: idx for idx, author in enumerate(self.authors)}
        self.style_to_idx = {style: idx for idx, style in enumerate(self.styles)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['path'].replace('\\', '/')
        full_path = os.path.join(self.img_dir, os.path.basename(img_path))
        image = Image.open(full_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        author = self.author_to_idx[self.data.iloc[idx]['artist']]
        style = self.style_to_idx[self.data.iloc[idx]['style']]
        return image, author, style

In [3]:
# Cargar datos
# Obtener la ruta absoluta del directorio donde está el script .py
script_dir = os.path.dirname(os.path.abspath("1001_images"))

# Ruta a la carpeta de imágenes que está en el mismo lugar que el script
img_dir = os.path.join(script_dir, "1001_images")
csv_path = "https://raw.githubusercontent.com/jsantonjag/PaintingsAI/refs/heads/main/data/dataset_completo.csv"

df = pd.read_csv(csv_path)
df['path'] = df['path'].apply(lambda x: os.path.join(img_dir, x))

full_dataset = ArtDataset(csv_file=csv_path, img_dir=img_dir, transform=transform)

# Dividir en train/test pequeño
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

## ResNet18 (ReLu, BatchNorm, Dropout, Optimizer-Adam, Scheduler)

In [4]:
# Modelo multitarea configurable
class MultiTaskResNet(nn.Module):
    def __init__(self, num_authors, num_styles, BATCHNORM, ACTIVATION_FN, DROPOUT, DROPOUT_PROB, LINK_FN, loss_function):
        super(MultiTaskResNet, self).__init__()
        
        self.loss_FN = loss_function
        self.link_FN = LINK_FN
        self.activation_FN = ACTIVATION_FN
        self.batchNorm = BATCHNORM
        self.dropout = DROPOUT
        self.dropout_prob = DROPOUT_PROB
        
        base_model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        in_features = base_model.fc.in_features

        layers = [nn.Flatten()]
        if self.batchNorm:
            layers.append(nn.BatchNorm1d(in_features))
            
        layers.append(self.wrap_activation(self.activation_FN))
        
        if self.dropout:
            layers.append(nn.Dropout(self.dropout_prob))
        
        layers.append(nn.Linear(in_features, in_features))
        self.shared_head = nn.Sequential(*layers)
        self.fc_artist = nn.Linear(in_features, num_authors)
        self.fc_style = nn.Linear(in_features, num_styles)

    def wrap_activation(self, ACTIVATION_FN):
        if ACTIVATION_FN == torch.relu:
            return nn.ReLU()
        elif ACTIVATION_FN == torch.tanh:
            return nn.Tanh()
        else:
            raise ValueError("Función de activación no soportada. Usa torch.relu o torch.tanh.")


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.shared_head(x)
        artist_output = self.link_FN(self.fc_artist(x), dim=1)
        style_output = self.link_FN(self.fc_style(x), dim=1)
        return artist_output, style_output
    
    def compute_loss(self, artist_logits, artist_targets, style_logits, style_targets):
        return self.loss_FN(artist_logits, artist_targets) + self.loss_FN(style_logits, style_targets)
    
    
def test_model(model, epochs, train_loader, test_loader, device):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loss_list, train_acc_artist_list, test_acc_artist_list = [], [], []
    train_acc_style_list, test_acc_style_list = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_artist, correct_style, total = 0, 0, 0

        for images, artist_labels, style_labels in train_loader:
            images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
            optimizer.zero_grad()
            artist_logits, style_logits = model(images)
            loss = model.compute_loss(artist_logits, artist_labels, style_logits, style_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            with torch.no_grad():
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist += (artist_preds == artist_labels).sum().item()
                correct_style += (style_preds == style_labels).sum().item()
                total += images.size(0)

        train_loss_list.append(total_loss / len(train_loader))
        train_acc_artist_list.append(correct_artist / total)
        train_acc_style_list.append(correct_style / total)

        # Evaluación en test
        model.eval()
        correct_artist_test, correct_style_test, total_test = 0, 0, 0
        with torch.no_grad():
            for images, artist_labels, style_labels in test_loader:
                images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
                artist_logits, style_logits = model(images)
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist_test += (artist_preds == artist_labels).sum().item()
                correct_style_test += (style_preds == style_labels).sum().item()
                total_test += images.size(0)

        test_acc_artist_list.append(correct_artist_test / total_test)
        test_acc_style_list.append(correct_style_test / total_test)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(train_loader):.4f} - Train Acc Artist: {correct_artist / total:.2%} - Train Acc Style: {correct_style / total:.2%}")

    print("=" * 50)
    print("Resultados finales de test:")
    print(f"Test Accuracy - Artista: {test_acc_artist_list[-1]*100:.2f}%")
    print(f"Test Accuracy - Estilo:  {test_acc_style_list[-1]*100:.2f}%")
    print("=" * 50)


    return train_loss_list, train_acc_artist_list, test_acc_artist_list, train_acc_style_list, test_acc_style_list

    

### Comparing activation functions (ReLU VS Tahn)

In [5]:
# ReLU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiTaskResNet(
    num_authors=len(full_dataset.authors), 
    num_styles=len(full_dataset.styles),
    BATCHNORM = True, 
    ACTIVATION_FN = torch.relu, 
    DROPOUT = True, 
    DROPOUT_PROB = 0.3, 
    LINK_FN = torch.softmax, 
    loss_function = nn.CrossEntropyLoss(),
)

losses, train_artist_acc, test_artist_acc, train_style_acc, test_style_acc = test_model(
    model=model,
    epochs=10,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device
)

Epoch 1/10 - Loss: 9.1597 - Train Acc Artist: 2.38% - Train Acc Style: 18.75%
Epoch 2/10 - Loss: 9.1515 - Train Acc Artist: 3.00% - Train Acc Style: 17.88%
Epoch 3/10 - Loss: 9.1508 - Train Acc Artist: 3.50% - Train Acc Style: 17.75%
Epoch 4/10 - Loss: 9.1446 - Train Acc Artist: 3.12% - Train Acc Style: 18.62%
Epoch 5/10 - Loss: 9.1617 - Train Acc Artist: 3.38% - Train Acc Style: 17.00%
Epoch 6/10 - Loss: 9.1702 - Train Acc Artist: 3.12% - Train Acc Style: 16.00%
Epoch 7/10 - Loss: 9.1718 - Train Acc Artist: 3.50% - Train Acc Style: 15.50%
Epoch 8/10 - Loss: 9.1695 - Train Acc Artist: 2.88% - Train Acc Style: 16.38%
Epoch 9/10 - Loss: 9.1617 - Train Acc Artist: 2.88% - Train Acc Style: 17.12%
Epoch 10/10 - Loss: 9.1685 - Train Acc Artist: 2.75% - Train Acc Style: 16.62%
Resultados finales de test:
Test Accuracy - Artista: 2.99%
Test Accuracy - Estilo:  16.92%


### Improving MultiTaskResNet 
* modifiying compute_loss
* adding scheduler
* dropout_prob=0.4
* removing softmax as link_function due to the loss_function (crossEntropyLoss()) has logsoftmax() in it.


In [6]:
# Modelo multitarea configurable
class MultiTaskResNet_1(nn.Module):
    def __init__(self, num_authors, num_styles, BATCHNORM, ACTIVATION_FN, DROPOUT, DROPOUT_PROB, loss_function):
        super(MultiTaskResNet_1, self).__init__()
        
        self.loss_FN = loss_function
        self.link_FN = lambda x, dim: x
        self.activation_FN = ACTIVATION_FN
        self.batchNorm = BATCHNORM
        self.dropout = DROPOUT
        self.dropout_prob = DROPOUT_PROB
        
        base_model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        in_features = base_model.fc.in_features

        layers = [nn.Flatten()]
        if self.batchNorm:
            layers.append(nn.BatchNorm1d(in_features))
        
        layers.append(nn.Linear(in_features, in_features))
        layers.append(self.wrap_activation(self.activation_FN))
        
        if self.dropout:
            layers.append(nn.Dropout(self.dropout_prob))
        
        layers.append(nn.Linear(in_features, in_features))
        layers.append(self.wrap_activation(self.activation_FN))
        
        self.shared_head = nn.Sequential(*layers)
        self.fc_artist = nn.Linear(in_features, num_authors)
        self.fc_style = nn.Linear(in_features, num_styles)

    def wrap_activation(self, ACTIVATION_FN):
        if ACTIVATION_FN == torch.relu:
            return nn.ReLU()
        elif ACTIVATION_FN == torch.tanh:
            return nn.Tanh()
        else:
            raise ValueError("Función de activación no soportada. Usa torch.relu o torch.tanh.")


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.shared_head(x)
        artist_output = self.link_FN(self.fc_artist(x), dim=1)
        style_output = self.link_FN(self.fc_style(x), dim=1)
        return artist_output, style_output
    
    def compute_loss(self, artist_logits, artist_targets, style_logits, style_targets):
        # ponderación opcional si una tarea es más difícil que otra
        return 0.6 * self.loss_FN(artist_logits, artist_targets) + 0.4 * self.loss_FN(style_logits, style_targets)
    
def test_model(model, epochs, train_loader, test_loader, device):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    
    train_loss_list, train_acc_artist_list, test_acc_artist_list = [], [], []
    train_acc_style_list, test_acc_style_list = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_artist, correct_style, total = 0, 0, 0

        for images, artist_labels, style_labels in train_loader:
            images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
            optimizer.zero_grad()
            artist_logits, style_logits = model(images)
            loss = model.compute_loss(artist_logits, artist_labels, style_logits, style_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            with torch.no_grad():
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist += (artist_preds == artist_labels).sum().item()
                correct_style += (style_preds == style_labels).sum().item()
                total += images.size(0)

        #Scheduling learning rate
        scheduler.step()
        
        train_loss_list.append(total_loss / len(train_loader))
        train_acc_artist_list.append(correct_artist / total)
        train_acc_style_list.append(correct_style / total)

        # Evaluación en test
        model.eval()
        correct_artist_test, correct_style_test, total_test = 0, 0, 0
        with torch.no_grad():
            for images, artist_labels, style_labels in test_loader:
                images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
                artist_logits, style_logits = model(images)
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist_test += (artist_preds == artist_labels).sum().item()
                correct_style_test += (style_preds == style_labels).sum().item()
                total_test += images.size(0)

        test_acc_artist_list.append(correct_artist_test / total_test)
        test_acc_style_list.append(correct_style_test / total_test)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(train_loader):.4f} - Train Acc Artist: {correct_artist / total:.2%} - Train Acc Style: {correct_style / total:.2%}")

    print("=" * 50)
    print("Resultados finales de test:")
    print(f"Test Accuracy - Artista: {test_acc_artist_list[-1]*100:.2f}%")
    print(f"Test Accuracy - Estilo:  {test_acc_style_list[-1]*100:.2f}%")
    print("=" * 50)


    return train_loss_list, train_acc_artist_list, test_acc_artist_list, train_acc_style_list, test_acc_style_list

    

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiTaskResNet_1(
    num_authors=len(full_dataset.authors), 
    num_styles=len(full_dataset.styles),
    BATCHNORM = True, 
    ACTIVATION_FN = torch.relu, 
    DROPOUT = True, 
    DROPOUT_PROB = 0.4,  
    loss_function = nn.CrossEntropyLoss(),
)

losses_1, train_artist_acc_1, test_artist_acc_1, train_style_acc_1, test_style_acc_1 = test_model(
    model=model,
    epochs=10,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device
)

Epoch 1/10 - Loss: 4.7360 - Train Acc Artist: 3.75% - Train Acc Style: 15.75%
Epoch 2/10 - Loss: 4.4289 - Train Acc Artist: 4.50% - Train Acc Style: 19.50%
Epoch 3/10 - Loss: 4.2898 - Train Acc Artist: 5.62% - Train Acc Style: 18.38%
Epoch 4/10 - Loss: 4.1257 - Train Acc Artist: 5.50% - Train Acc Style: 19.88%
Epoch 5/10 - Loss: 3.9894 - Train Acc Artist: 6.25% - Train Acc Style: 21.88%
Epoch 6/10 - Loss: 3.8906 - Train Acc Artist: 7.25% - Train Acc Style: 23.25%
Epoch 7/10 - Loss: 3.7576 - Train Acc Artist: 6.88% - Train Acc Style: 21.62%
Epoch 8/10 - Loss: 3.6540 - Train Acc Artist: 7.25% - Train Acc Style: 25.00%
Epoch 9/10 - Loss: 3.5791 - Train Acc Artist: 7.50% - Train Acc Style: 26.25%
Epoch 10/10 - Loss: 3.3903 - Train Acc Artist: 11.00% - Train Acc Style: 29.88%
Resultados finales de test:
Test Accuracy - Artista: 4.48%
Test Accuracy - Estilo:  25.37%


### Improving MultiTaskResNet (by using ResNet34)

In [8]:
from torchvision.models import resnet34, ResNet34_Weights

In [9]:
# Modelo multitarea configurable
class MultiTaskResNet34(nn.Module):
    def __init__(self, num_authors, num_styles, BATCHNORM, ACTIVATION_FN, DROPOUT, DROPOUT_PROB, loss_function):
        super(MultiTaskResNet34, self).__init__()
        
        self.loss_FN = loss_function
        self.link_FN = lambda x, dim: x
        self.activation_FN = ACTIVATION_FN
        self.batchNorm = BATCHNORM
        self.dropout = DROPOUT
        self.dropout_prob = DROPOUT_PROB
        
        base_model = resnet34(weights=ResNet34_Weights.DEFAULT)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        in_features = base_model.fc.in_features

        layers = [nn.Flatten()]
        if self.batchNorm:
            layers.append(nn.BatchNorm1d(in_features))
            
        layers += [
            nn.Linear(in_features, 512),
            self.wrap_activation(self.activation_FN),
        ]
                
        if self.dropout:
            layers.append(nn.Dropout(self.dropout_prob))
        
        layers += [
            nn.Linear(512, 256),
            self.wrap_activation(self.activation_FN),
        ]
        
        if self.dropout:
            layers.append(nn.Dropout(self.dropout_prob))
        
        layers += [
            nn.Linear(256, in_features),
            self.wrap_activation(self.activation_FN),
        ]
        
        self.shared_head = nn.Sequential(*layers)
        self.fc_artist = nn.Linear(in_features, num_authors)
        self.fc_style = nn.Linear(in_features, num_styles)

    def wrap_activation(self, ACTIVATION_FN):
        if ACTIVATION_FN == torch.relu:
            return nn.ReLU()
        elif ACTIVATION_FN == torch.tanh:
            return nn.Tanh()
        else:
            raise ValueError("Función de activación no soportada. Usa torch.relu o torch.tanh.")


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.shared_head(x)
        artist_output = self.link_FN(self.fc_artist(x), dim=1)
        style_output = self.link_FN(self.fc_style(x), dim=1)
        return artist_output, style_output
    
    def compute_loss(self, artist_logits, artist_targets, style_logits, style_targets):
        # ponderación opcional si una tarea es más difícil que otra
        return 0.6 * self.loss_FN(artist_logits, artist_targets) + 0.4 * self.loss_FN(style_logits, style_targets)
    
def test_model(model, epochs, train_loader, test_loader, device):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    
    train_loss_list, train_acc_artist_list, test_acc_artist_list = [], [], []
    train_acc_style_list, test_acc_style_list = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_artist, correct_style, total = 0, 0, 0

        for images, artist_labels, style_labels in train_loader:
            images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
            optimizer.zero_grad()
            artist_logits, style_logits = model(images)
            loss = model.compute_loss(artist_logits, artist_labels, style_logits, style_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            with torch.no_grad():
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist += (artist_preds == artist_labels).sum().item()
                correct_style += (style_preds == style_labels).sum().item()
                total += images.size(0)

        #Scheduling learning rate
        scheduler.step()
        
        train_loss_list.append(total_loss / len(train_loader))
        train_acc_artist_list.append(correct_artist / total)
        train_acc_style_list.append(correct_style / total)

        # Evaluación en test
        model.eval()
        correct_artist_test, correct_style_test, total_test = 0, 0, 0
        with torch.no_grad():
            for images, artist_labels, style_labels in test_loader:
                images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
                artist_logits, style_logits = model(images)
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist_test += (artist_preds == artist_labels).sum().item()
                correct_style_test += (style_preds == style_labels).sum().item()
                total_test += images.size(0)

        test_acc_artist_list.append(correct_artist_test / total_test)
        test_acc_style_list.append(correct_style_test / total_test)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(train_loader):.4f} - Train Acc Artist: {correct_artist / total:.2%} - Train Acc Style: {correct_style / total:.2%}")

    print("=" * 50)
    print("Resultados finales de test:")
    print(f"Test Accuracy - Artista: {test_acc_artist_list[-1]*100:.2f}%")
    print(f"Test Accuracy - Estilo:  {test_acc_style_list[-1]*100:.2f}%")
    print("=" * 50)

    return train_loss_list, train_acc_artist_list, test_acc_artist_list, train_acc_style_list, test_acc_style_list

    

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiTaskResNet34(
    num_authors=len(full_dataset.authors), 
    num_styles=len(full_dataset.styles),
    BATCHNORM = True, 
    ACTIVATION_FN = torch.relu, 
    DROPOUT = True, 
    DROPOUT_PROB = 0.4,  
    loss_function = nn.CrossEntropyLoss(),
)

losses_36, train_artist_acc_36, test_artist_acc_36, train_style_acc_36, test_style_acc_36 = test_model(
    model=model,
    epochs=10,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device
)

Epoch 1/10 - Loss: 4.7573 - Train Acc Artist: 3.25% - Train Acc Style: 13.00%
Epoch 2/10 - Loss: 4.5524 - Train Acc Artist: 3.38% - Train Acc Style: 14.50%
Epoch 3/10 - Loss: 4.5387 - Train Acc Artist: 3.00% - Train Acc Style: 15.62%
Epoch 4/10 - Loss: 4.5072 - Train Acc Artist: 3.38% - Train Acc Style: 16.00%
Epoch 5/10 - Loss: 4.4854 - Train Acc Artist: 3.12% - Train Acc Style: 16.88%
Epoch 6/10 - Loss: 4.4393 - Train Acc Artist: 3.75% - Train Acc Style: 17.62%
Epoch 7/10 - Loss: 4.4133 - Train Acc Artist: 3.50% - Train Acc Style: 17.38%
Epoch 8/10 - Loss: 4.3890 - Train Acc Artist: 3.38% - Train Acc Style: 16.25%
Epoch 9/10 - Loss: 4.3247 - Train Acc Artist: 3.88% - Train Acc Style: 17.50%
Epoch 10/10 - Loss: 4.2484 - Train Acc Artist: 4.38% - Train Acc Style: 18.50%
Resultados finales de test:
Test Accuracy - Artista: 3.48%
Test Accuracy - Estilo:  17.91%


### Improving ResNet34 
* Modifiying dropout_prob (dropout_prob=0.2), resize (224x224), compute_loss (70% loss-artist and 30% loss-style)
* Grouping artists with fewer than 10 pictures
* Applying argumentations in transformers (rotation, flip, colorjitter)
* Balanced classes (with WeightedRandomSampler()): calculating inverse weights, balanced sampling in training with inverse weights.
* Adding task-specific heads (self.artist_head & self.style_head)
* Modifiying scheduler ("ReduceLROnPlateau") para ajustar el learning rate
* Adding epochs (epochs = 20)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt

In [12]:
# Transformaciones
transform_ = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Dataset personalizado
class ArtDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.authors = sorted(self.data['artist'].unique())
        self.styles = sorted(self.data['style'].unique())
        self.author_to_idx = {author: idx for idx, author in enumerate(self.authors)}
        self.style_to_idx = {style: idx for idx, style in enumerate(self.styles)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['path'].replace('\\', '/')
        full_path = os.path.join(self.img_dir, os.path.basename(img_path))
        image = Image.open(full_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        author = self.author_to_idx[self.data.iloc[idx]['artist']]
        style = self.style_to_idx[self.data.iloc[idx]['style']]
        return image, author, style

# Modelo multitarea configurable
class MultiTaskResNet_m(nn.Module):
    def __init__(self, num_authors, num_styles, BATCHNORM, ACTIVATION_FN, DROPOUT, DROPOUT_PROB, loss_function):
        super(MultiTaskResNet_m, self).__init__()
        
        self.loss_FN = loss_function
        self.link_FN = lambda x, dim: x
        self.activation_FN = ACTIVATION_FN
        self.batchNorm = BATCHNORM
        self.dropout = DROPOUT
        self.dropout_prob = DROPOUT_PROB
        
        base_model = resnet34(weights=ResNet34_Weights.DEFAULT)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        in_features = base_model.fc.in_features

        layers = [nn.Flatten()]
        if self.batchNorm:
            layers.append(nn.BatchNorm1d(in_features))
        
        layers.append(nn.Linear(in_features, in_features))
        layers.append(self.wrap_activation(self.activation_FN))
        
        if self.dropout:
            layers.append(nn.Dropout(self.dropout_prob))
        
        layers.append(nn.Linear(in_features, in_features))
        layers.append(self.wrap_activation(self.activation_FN))
        
        self.shared_head = nn.Sequential(*layers)
        self.fc_artist = nn.Linear(in_features, num_authors)
        self.artist_head = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(),
            nn.Dropout(self.dropout_prob)
        )
        self.style_head = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(),
            nn.Dropout(self.dropout_prob)
        )
        self.fc_style = nn.Linear(in_features, num_styles)

    def wrap_activation(self, ACTIVATION_FN):
        if ACTIVATION_FN == torch.relu:
            return nn.ReLU()
        elif ACTIVATION_FN == torch.tanh:
            return nn.Tanh()
        else:
            raise ValueError("Función de activación no soportada. Usa torch.relu o torch.tanh.")


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.shared_head(x)
        
        artist_output = self.link_FN(self.fc_artist(self.artist_head(x)), dim=1)
        style_output = self.link_FN(self.fc_style(self.style_head(x)), dim=1)
        
        return artist_output, style_output
    
    def compute_loss(self, artist_logits, artist_targets, style_logits, style_targets):
        # ponderación opcional si una tarea es más difícil que otra
        return 0.7 * self.loss_FN(artist_logits, artist_targets) + 0.3 * self.loss_FN(style_logits, style_targets)
    
def test_model(model, epochs, train_loader, test_loader, device):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    
    train_loss_list, train_acc_artist_list, test_acc_artist_list = [], [], []
    train_acc_style_list, test_acc_style_list = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_artist, correct_style, total = 0, 0, 0

        for images, artist_labels, style_labels in train_loader:
            images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
            optimizer.zero_grad()
            artist_logits, style_logits = model(images)
            loss = model.compute_loss(artist_logits, artist_labels, style_logits, style_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            with torch.no_grad():
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist += (artist_preds == artist_labels).sum().item()
                correct_style += (style_preds == style_labels).sum().item()
                total += images.size(0)

               
        train_loss_list.append(total_loss / len(train_loader))
        train_acc_artist_list.append(correct_artist / total)
        train_acc_style_list.append(correct_style / total)

        # Evaluación en test
        model.eval()
        correct_artist_test, correct_style_test, total_test = 0, 0, 0
        with torch.no_grad():
            for images, artist_labels, style_labels in test_loader:
                images, artist_labels, style_labels = images.to(device), artist_labels.to(device), style_labels.to(device)
                artist_logits, style_logits = model(images)
                artist_preds = torch.argmax(artist_logits, dim=1)
                style_preds = torch.argmax(style_logits, dim=1)
                correct_artist_test += (artist_preds == artist_labels).sum().item()
                correct_style_test += (style_preds == style_labels).sum().item()
                total_test += images.size(0)
        
        test_acc_artist_list.append(correct_artist_test / total_test)
        test_acc_style_list.append(correct_style_test / total_test)

        scheduler.step(correct_artist_test/total_test)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(train_loader):.4f} - Train Acc Artist: {correct_artist / total:.2%} - Train Acc Style: {correct_style / total:.2%}")

    print("=" * 50)
    print("Resultados finales de test:")
    print(f"Test Accuracy - Artista: {test_acc_artist_list[-1]*100:.2f}%")
    print(f"Test Accuracy - Estilo:  {test_acc_style_list[-1]*100:.2f}%")
    print("=" * 50)

    return train_loss_list, train_acc_artist_list, test_acc_artist_list, train_acc_style_list, test_acc_style_list

    

In [13]:
# Preparar datos
script_dir = os.path.dirname(os.path.abspath("1001_images"))
img_dir = os.path.join(script_dir, "1001_images")
csv_path = "https://raw.githubusercontent.com/jsantonjag/PaintingsAI/refs/heads/main/data/dataset_completo.csv"
df = pd.read_csv(csv_path)

#Agrupar artistas con <10 imágenes como "Otros"
artist_counts = df['artist'].value_counts()
valid_artists = artist_counts[artist_counts >= 10].index.tolist()
df['artist'] = df['artist'].apply(lambda x: x if x in valid_artists else 'Otros')

df.to_csv("filtered_dataset.csv", index=False)

df['path'] = df['path'].apply(lambda x: os.path.join(img_dir, x))

full_dataset = ArtDataset(csv_file="filtered_dataset.csv", img_dir=img_dir, transform=transform_)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Calcular pesos inversos por clase
author_labels = [sample[1] for sample in train_dataset]
class_sample_counts = pd.Series(author_labels).value_counts().sort_index()
weights = 1. / class_sample_counts
sample_weights = [weights[label] for label in author_labels]

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=16)


In [14]:
#Entrenamiento con la ReLU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiTaskResNet_m(
    num_authors=len(full_dataset.authors), 
    num_styles=len(full_dataset.styles),
    BATCHNORM = True, 
    ACTIVATION_FN = torch.relu, 
    DROPOUT = True, 
    DROPOUT_PROB = 0.2,  
    loss_function = nn.CrossEntropyLoss(),
)

losses_n, train_artist_acc_n, test_artist_acc_n, train_style_acc_n, test_style_acc_n = test_model(
    model=model,
    epochs=20,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device
)

Epoch 1/20 - Loss: 2.3403 - Train Acc Artist: 22.00% - Train Acc Style: 37.62%
Epoch 2/20 - Loss: 2.1987 - Train Acc Artist: 25.00% - Train Acc Style: 39.25%
Epoch 3/20 - Loss: 2.2069 - Train Acc Artist: 22.75% - Train Acc Style: 38.12%
Epoch 4/20 - Loss: 2.0347 - Train Acc Artist: 28.38% - Train Acc Style: 38.75%
Epoch 5/20 - Loss: 1.9232 - Train Acc Artist: 33.12% - Train Acc Style: 47.88%
Epoch 6/20 - Loss: 1.8718 - Train Acc Artist: 35.12% - Train Acc Style: 49.12%
Epoch 7/20 - Loss: 1.8299 - Train Acc Artist: 36.62% - Train Acc Style: 49.88%
Epoch 8/20 - Loss: 1.5828 - Train Acc Artist: 43.88% - Train Acc Style: 56.75%
Epoch 9/20 - Loss: 1.5297 - Train Acc Artist: 47.38% - Train Acc Style: 56.88%
Epoch 10/20 - Loss: 1.3384 - Train Acc Artist: 52.38% - Train Acc Style: 62.00%
Epoch 11/20 - Loss: 1.2891 - Train Acc Artist: 56.12% - Train Acc Style: 64.50%
Epoch 12/20 - Loss: 1.2284 - Train Acc Artist: 57.50% - Train Acc Style: 65.62%
Epoch 13/20 - Loss: 1.0735 - Train Acc Artist: 62